In [49]:
from aocd.models import Puzzle
import json
import operator
puzzle = Puzzle(year=2023, day=19)
data = puzzle.input_data
%run helper.ipynb

In [53]:
class Rule(object):
    def __init__(self, s):
        self.default_rule = False
        self.category = None
        self.func = None
        self.value = None
        if ":" not in s:
            self.default_rule = True
            self.result = s
            self.range = range(1, 4001)
        else:
            rule, result = s.split(":")
            self.result = result
            self.category = rule[0]
            self.value = int(rule[2:])
            if rule[1] == "<":
                self.func = operator.lt
                self.range = range(1, self.value)
            elif rule[1] == ">":
                self.func = operator.gt
                self.range = range(self.value + 1, 4001)

    def evaluate(self, part):
        if self.default_rule:
            return self.result
        if self.func(part[self.category], self.value):
            return self.result
        return ""

class Workflow(object):
    def __init__(self, s):
        name, remainder = s.split("{")
        self.name = name
        remainder = remainder.strip("}")
        self.rules = [Rule(r) for r in remainder.split(",")]

    def evaluate(self, part, other_workflows):
        for rule in self.rules:
            result = rule.evaluate(part)
            if result == "":
                continue
            elif result == "A":
                return True
            elif result == "R":
                return False
            elif result in other_workflows:
                return other_workflows[result].evaluate(part, other_workflows)
            else:
                print(part, result, rule.category, rule.value, rule.func, "ERROR")

def get_new_ranges(old_r, new_r):
    matching = range(max(old_r.start, new_r.start), min(old_r.stop, new_r.stop))
    if old_r.start < new_r.start:
        not_matching = range(old_r.start, min(old_r.stop, new_r.start))
    else:
        not_matching = range(max(old_r.start, new_r.stop), old_r.stop)
    return matching, not_matching

def get_accepted_ranges(curr_ranges, workflow):
#     print(workflow.name)
    ret_ranges = []
    for rule in workflow.rules:
        if rule.default_rule:
            if rule.result == "A":
                ret_ranges.append(curr_ranges)
            elif rule.result == "R":
                continue
            else:
                ret_ranges += get_accepted_ranges(curr_ranges, workflow_dict[rule.result])
        else:
            matching, not_matching = get_new_ranges(curr_ranges[rule.category], rule.range)
            if len(matching) > 0:
                if rule.result == "A":
                    ret_ranges.append(curr_ranges)
                elif rule.result == "R":
                    continue
                else:
                    ret_ranges += get_accepted_ranges(curr_ranges | {rule.category: matching}, workflow_dict[rule.result])
            if not_matching:
                curr_ranges = curr_ranges | {rule.category: not_matching}
#     print("base", ret_ranges)
    return ret_ranges

In [54]:
workflows, parts = data.split("\n\n")
parts = parts.replace('x', '"x"').replace('m', '"m"').replace('a', '"a"').replace('s', '"s"').replace('=', ':')
workflows = workflows.split("\n")
parts = [json.loads(x) for x in parts.split("\n")]
workflows = [Workflow(x) for x in workflows]
workflow_dict = {w.name: w for w in workflows}

In [4]:
good_parts = [part for part in parts if workflow_dict["in"].evaluate(part, workflow_dict)]

In [5]:
puzzle.answer_a = sum([sum(list(part.values())) for part in good_parts])

In [11]:
start_ranges = {"x": range(1,4001), "m": range(1,4001), "a": range(1,4001), "s": range(1,4001)}

In [55]:
rrs = get_accepted_ranges(start_ranges, workflow_dict["in"])

In [ ]:
# 132392981697081